In [45]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd

In [46]:
df = pd.read_csv('./raw-osm-data.csv')

In [47]:
# keep rows have at least 3 elements
df = df.dropna(thresh=3)
df.shape

(155049, 13)

In [48]:
# filter address have road is no empty
df = df[df.road.notna()]

### Scale down columns

In [49]:
def checkTWCity(df):
    if df.any() != None:
        if pd.isna(df.state) and pd.isna(df.region) :
            try:
                if df.city == 'Thành phố Thủ Đức':
                    df.state = 'Thành phố Hồ Chí Minh'
                    return df
                if 'Thành phố' in df.city:
                    df.state = df.city
                    df.city = float('nan')
            except:
                pass
        elif pd.isna(df.state):
            df.state = df.region

        return df

df = df.apply(checkTWCity, axis=1)
df = df.dropna(subset=['state'])
df = df.drop(columns = ['region'])
df = df.drop_duplicates(
  subset = ['state','city','county','district','city_district','town','suburb','village','neighbourhood','road'],
  keep = 'first').reset_index(drop = True)
df.shape

(74191, 12)

"Sân cỏ nhân tạo Học Viện, Ngõ 86 Đường Trâu Quỳ, Thị trấn Trâu Quỳ, Huyện Gia Lâm, Thành phố Hà Nội, 1310, Việt Nam"  -> "Ngõ 86 Đường Trâu Quỳ, Thị trấn Trâu Quỳ, Huyện Gia Lâm, Thành phố Hà Nội"

In [50]:
def shortcut_address(df):
    if df.any != None:
        if df.address:
            add_list = df.address.split(',')

            index = str(df.address).find(str(df.road))

            index2 = len(df.address)
            add_list[-2] = str(add_list[-2]).strip()
            if str(add_list[-2]).isnumeric():
                index2 = str(df.address).find(str(add_list[-2]))-1
            elif str(add_list[-2]).find(str('-')) != -1 and str(add_list[-2]) != 'Tỉnh Bà Rịa-Vũng Tàu':
                index2 = str(df.address).find(str(add_list[-2]))-1
            elif str(add_list[-2]).find(str('+')) != -1 :
                index2 = str(df.address).find(str(add_list[-2]))-1
            add_list[-1] = str(add_list[-1]).strip()
            if str(add_list[-1]) == 'Việt Nam' and  index2 == len(df.address):
                index2 = str(df.address).find(str(add_list[-1]))-1
            if index != -1:
                df.address = df.address[index:index2].strip()
                df.address = df.address.strip(',')
                if df.address.count(',') > 3:
                    df['status'] = 'more'
                else:
                    df['status'] = 'less'
                    

    return df

#### df = df_less (<= 4 fields)  +  df_more (others)

In [51]:
df = df.apply(shortcut_address, axis=1)
df_less = df[df.status == 'less']
df_more = df[df.status == 'more']

In [52]:
df = df_less

In [53]:
def rm_ifExist_s(df, s):
    if df != None:
        if pd.isna(df):
            return df
        if s in df: 
            # print(df,s)
            df = 'remove'
        
    return df


In [54]:
def replaceString(df,s,sChange):
    if df != None:
        index = df.find(s)
        if index != -1:
            return str(sChange + df[index + len(s) :]).strip()
    return df

In [55]:
df['road'] = df['road'].apply(replaceString,s='\\', sChange='')
df['road'] = df['road'].apply(replaceString,s='Đướng', sChange='Đường')

In [56]:
def rm_brackets(df):
    if df!= None:
        index = df.find('(')
        if index != -1:
            return df[:index].strip()
    return df

In [57]:
df.road = df.road.apply(rm_brackets)

In [58]:
df = df.dropna(subset=['road'])
df = df.drop(df[df.road == ''].index)
df = df.drop(df[df['road']  == 'Cầu'].index)

In [59]:
remove_road = ['Đường lên','đường lên','Đường Lên','Lên Trụ','Lên trụ','lên trụ','Xuống trụ','Vòng xoay','Vòng Xoay','Phố nhỏ','Thôn','Đường vào','đường vào','turn','xe máy','Đường tránh','Đường Tránh','cao tốc','ĐH Ngô Gia Tự',
                'song hành','Song Hành','hàng cây','Sân','sân','Đường Ga','Bridge','bridge','-','+','Ngã tư','hầm','Hầm','Đèo','road','Road','ven biển','RAI67_YY_005','RAI','bờ ruộng','Bờ ruộng','ven ','Ven ','Giữa ',
                'giữa ','dân sinh','đường liên','đường Liên','Đường Liên','vượt','Vượt','dẫn','path','khu đô thị','Khu Đô Thị','KĐT','vào','Vào','Trên Núi','Trung tâm','Thõn','Thòn','Rd','rd','Internal','khu ','Khu '
                ,'Khu Dân Cư','khu dân cư','KDC','trồng','loi vao','lối','Thủy Điện','thủy điện','xả lũ','cũ ','Cũ ','Duong dat','duong dat','Đường đất','đường đất','Traại','Trail','trail','Trek','sidewalk','cầu',
                'Cầu treo','Cầu treo','tuần tra','Tuần Tra','Đường Dọc','đường dọc','Đường dọc','bypass','Đường ra ','đường ra','Đường qua ','Không tên','Không Tên','Cave','liên xóm','Liên Xóm','Đường trong','Đường đi ',
                'Đường Đi ','đường đi ','liên xã','Liên Xã','đi bộ','Đi Bộ','Lộ tẻ','no name','No Name','Vòng xuyến','Vòng Xuyến','bao biển','into','hướng đi','Hướng đi']

In [60]:
for s in remove_road:
    df.road = df.road.apply(rm_ifExist_s, s=s)

In [61]:
df_remove = df[df.road == 'remove']
df = df.drop(df[df.road == 'remove'].index)

In [62]:
remove_ward = ['khu đô thị','Khu Đô Thị','KĐT','khu ','Khu ','Khu Dân Cư','khu dân cư']
for s in remove_ward:
    df.address = df.address.apply(rm_ifExist_s, s=s)

df_tmp = df[df.address == 'remove']
df_remove = df_remove.append(df_tmp)
df = df.drop(df[df.address == 'remove'].index)

In [63]:
df

,osm_id,state,city,county,district,city_district,town,suburb,village,neighbourhood,road,address,status
2,589659644,Thành phố Hà Nội,NaN,NaN,NaN,Huyện Gia Lâm,NaN,NaN,Thị trấn Trâu Quỳ,NaN,Ngõ 86 Đường Trâu Quỳ,"Ngõ 86 Đường Trâu Quỳ, Thị trấn Trâu Quỳ, Huyệ...",less
4,508344626,Thành phố Hồ Chí Minh,NaN,NaN,NaN,Quận Tân Phú,NaN,Phường Phú Thạnh,NaN,NaN,Đường Nguyễn Sơn,"Đường Nguyễn Sơn, Phường Phú Thạnh, Quận Tân P...",less
5,508741166,Thành phố Hồ Chí Minh,NaN,NaN,NaN,Quận Bình Tân,NaN,Phường An Lạc,NaN,NaN,Đường Kinh Dương Vương,"Đường Kinh Dương Vương, Phường An Lạc, Quận Bì...",less
6,509015321,Thành phố Hồ Chí Minh,NaN,NaN,NaN,Quận 10,NaN,Phường 11,NaN,NaN,Đường 3 Tháng 2,"Đường 3 Tháng 2, Phường 11, Quận 10, Thành phố...",less
7,509025836,Thành phố Hồ Chí Minh,NaN,NaN,NaN,Quận Tân Phú,NaN,Phường Tân Thới Hòa,NaN,NaN,Hẻm 67 Lũy Bán Bích,"Hẻm 67 Lũy Bán Bích, Phường Tân Thới Hòa, Quận...",less
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74180,582757803,Thành phố Hà Nội,NaN,NaN,NaN,Quận Long Biên,NaN,Phường Đức Giang,NaN,NaN,ngõ 53/81,"ngõ 53/81, Phường Đức Giang, Quận Long Biên, T...",less
74181,582806643,Thành phố Hà Nội,NaN,NaN,NaN,Quận Long Biên,NaN,Phường Đức Giang,NaN,NaN,17b Hẻm 486/14/10,"17b Hẻm 486/14/10, Phường Đức Giang, Quận Long...",less
74183,520932991,Thành phố Hồ Chí Minh,NaN,NaN,NaN,Quận Bình Tân,NaN,Phường Bình Trị Đông,NaN,NaN,Hẻm 1/29 Chiến Lược,"Hẻm 1/29 Chiến Lược, Phường Bình Trị Đông, Quậ...",less
74184,582757802,Thành phố Hà Nội,NaN,NaN,NaN,Quận Long Biên,NaN,Phường Đức Giang,NaN,NaN,hẻm 53/81/3,"hẻm 53/81/3, Phường Đức Giang, Quận Long Biên,...",less


In [64]:
# check number of ',' in address
def check_comma(df):
    if df.any != None:
        lst = df['address'].split(',')
        if len(lst)<3: 
            df['status'] = 'remove'
        elif len(lst)==3: 
            df['status'] = '3'
    return df

In [65]:
df = df.dropna(subset=['address'])
df = df.apply(check_comma, axis=1)
df.shape

(58813, 13)

In [66]:
df_3fields = df[df.status == '3']
df = df.drop(df[df.status == 'remove'].index)
df = df.drop(df[df.status == '3'].index)

df.to_csv('4fields.csv', index=False, header=True)
df_3fields.to_csv('3fields.csv', index=False, header=True)

df_remove = df_remove.append(df_3fields)
print("Total removed rows:",df_remove.shape)

Total removed rows: (15078, 13)


### config rows have 3 fields 

In [67]:
df_3fields

,osm_id,state,city,county,district,city_district,town,suburb,village,neighbourhood,road,address,status
25,587976967,Tỉnh Phú Thọ,Việt Trì,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nguyễn Tất Thành,"Nguyễn Tất Thành, Việt Trì, Tỉnh Phú Thọ",3
39,546126801,Tỉnh Cà Mau,NaN,Trần Văn Thời,NaN,NaN,Trần Văn Thời,NaN,NaN,NaN,Đường Dường Vàn Thà,"Đường Dường Vàn Thà, Trần Văn Thời, Tỉnh Cà Mau",3
42,570587154,Thành phố Hà Nội,NaN,NaN,NaN,Quận Cầu Giấy,NaN,NaN,NaN,NaN,Ngõ 169 Doãn Kế Thiện,"Ngõ 169 Doãn Kế Thiện, Quận Cầu Giấy, Thành ph...",3
48,559962867,Tỉnh Quảng Nam,Thành phố Tam Kỳ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nguyễn Tất Thành,"Nguyễn Tất Thành, Thành phố Tam Kỳ, Tỉnh Quảng...",3
65,584375627,Tỉnh Quảng Nam,NaN,Núi Thành,NaN,NaN,NaN,NaN,NaN,NaN,Old Runway 14/32,"Old Runway 14/32, Núi Thành, Tỉnh Quảng Nam",3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74115,568518814,Thành phố Hà Nội,NaN,NaN,NaN,Quận Cầu Giấy,NaN,NaN,NaN,NaN,Ngõ 2 Nguyễn Chánh,"Ngõ 2 Nguyễn Chánh, Quận Cầu Giấy, Thành phố H...",3
74127,567218542,Tỉnh Tây Ninh,Thành phố Tây Ninh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hẻm 25 Trần Hưng Đạo,"hẻm 25 Trần Hưng Đạo, Thành phố Tây Ninh, Tỉnh...",3
74147,525191975,Tỉnh Lâm Đồng,Đà Lạt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hẻm 29B Phan Bội Châu,"Hẻm 29B Phan Bội Châu, Đà Lạt, Tỉnh Lâm Đồng",3
74162,571634189,Tỉnh Tây Ninh,Thành phố Tây Ninh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hẻm 8 Ba Mươi Tháng Tư,"hẻm 8 Ba Mươi Tháng Tư, Thành phố Tây Ninh, Tỉ...",3


In [68]:
backup = df_3fields

In [69]:
remove_road_3fields = ['Runway','Transfer','minor','lên','Lên','KQH','đường mòn','Đường mòn','sửa','xóm','Địa Phận',';','Đường chính','Đường sắt','Tránh','tránh','Bờ Bắc','Bờ Nam','Bùng binh','–','Đền','NỘI BỘ','nội bộ','Nội Bộ','Đập','Đường xã','Hẻm chợ','liên Bản','Old','Path','beach']

In [70]:
for s in remove_road_3fields:
    df_3fields.road = df_3fields.road.apply(rm_ifExist_s, s=s)
    
tmp = df_3fields[df_3fields.road == 'remove']
df_remove = df_remove.append(tmp)
df_3fields = df_3fields.drop(df_3fields[df_3fields.road == 'remove'].index)

In [72]:
for st in [' 4x4','Phố mới ' ,'Đường thôn ', ' Street',' nối dài',' Nối dài', ' (Nối dài)']:
    df_3fields['road'] = df_3fields['road'].apply(replaceString,s=st, sChange='')

df_3fields = df_3fields.drop(df_3fields[df_3fields.road == 'ngõ'].index)

In [75]:
address_3fields = df_3fields['address'].to_list()
_3fields =pd.DataFrame(columns=['Street','Ward','District','Province'])

for s in address_3fields:
    address = s.split(',')
    tmp = pd.DataFrame([[address[0],'',address[1], address[2]]],
                      columns=['Street','Ward','District','Province'])
    _3fields = _3fields.append(tmp)

print(_3fields)

                        Street Ward             District           Province
0             Nguyễn Tất Thành                  Việt Trì       Tỉnh Phú Thọ
0          Đường Dường Vàn Thà             Trần Văn Thời        Tỉnh Cà Mau
0        Ngõ 169 Doãn Kế Thiện             Quận Cầu Giấy   Thành phố Hà Nội
0             Nguyễn Tất Thành          Thành phố Tam Kỳ     Tỉnh Quảng Nam
0       hẻm 6 Ba Mươi Tháng Tư        Thành phố Tây Ninh      Tỉnh Tây Ninh
..                         ...  ...                  ...                ...
0           Ngõ 2 Nguyễn Chánh             Quận Cầu Giấy   Thành phố Hà Nội
0         hẻm 25 Trần Hưng Đạo        Thành phố Tây Ninh      Tỉnh Tây Ninh
0        Hẻm 29B Phan Bội Châu                    Đà Lạt      Tỉnh Lâm Đồng
0       hẻm 8 Ba Mươi Tháng Tư        Thành phố Tây Ninh      Tỉnh Tây Ninh
0   Hẻm 25 Đường Trần Hưng Đạo             Buôn Ma Thuột            Đắk Lắk

[11891 rows x 4 columns]


In [76]:
address_4fields = df['address'].to_list()
_4fields =pd.DataFrame(columns=['Street','Ward','District','Province'])

for s in address_4fields:
    address = s.split(',')
    tmp = pd.DataFrame([[address[0],address[1], address[2], address[3]]],
                      columns=['Street','Ward','District','Province'])
    _4fields = _4fields.append(tmp, ignore_index=True)

df_final = _4fields
df_final = df_final.append(_3fields)

In [77]:
df_final = df_final.drop_duplicates(keep='first')
df_final.to_csv('DataOpenStreetMap.csv', index=False, header=True)

In [78]:
df_final

,Street,Ward,District,Province
0,Ngõ 86 Đường Trâu Quỳ,Thị trấn Trâu Quỳ,Huyện Gia Lâm,Thành phố Hà Nội
1,Đường Nguyễn Sơn,Phường Phú Thạnh,Quận Tân Phú,Thành phố Hồ Chí Minh
2,Đường Kinh Dương Vương,Phường An Lạc,Quận Bình Tân,Thành phố Hồ Chí Minh
3,Đường 3 Tháng 2,Phường 11,Quận 10,Thành phố Hồ Chí Minh
4,Hẻm 67 Lũy Bán Bích,Phường Tân Thới Hòa,Quận Tân Phú,Thành phố Hồ Chí Minh
...,...,...,...,...
0,Ngõ 2 Nguyễn Chánh,,Quận Cầu Giấy,Thành phố Hà Nội
0,hẻm 25 Trần Hưng Đạo,,Thành phố Tây Ninh,Tỉnh Tây Ninh
0,Hẻm 29B Phan Bội Châu,,Đà Lạt,Tỉnh Lâm Đồng
0,hẻm 8 Ba Mươi Tháng Tư,,Thành phố Tây Ninh,Tỉnh Tây Ninh
